# Análisis de datos y cálculo de distancia euclídea
## 1. Importación de librerías y establecimiento de directorio de trabajo

In [1]:
import os
import geopandas as gpd
import requests
import urllib.parse
import warnings
warnings.simplefilter("ignore")
import time
import random
import haversine as hs

os.chdir("C:/Users/aleja/OneDrive - Universidad Politécnica de Madrid/Universidad/Asignaturas/Cuarto/Trabajo de Fin de Grado/")

## 2. Carga de datos

In [2]:
accesos = gpd.read_file("Capas/ACCESOS.geojson")
actividades_economicas = gpd.read_file("Capas/ACTIVIDADES.geojson")
administracion = gpd.read_file("Capas/POIS/poi_administracio_ca.geojson")
cultura = gpd.read_file("Capas/POIS/poi_cultura_ca.geojson")
educacion = gpd.read_file("Capas/POIS/poi_educacio_ca.geojson")
deporte = gpd.read_file("Capas/POIS/poi_esports_ca.geojson")
ocio = gpd.read_file("Capas/POIS/poi_lloi_ca.geojson")
sanidad = gpd.read_file("Capas/POIS/poi_sanitat_ca.geojson")
servicios = gpd.read_file("Capas/POIS/poi_serveis_ca.geojson")
movilidad = gpd.read_file("Capas/POIS/poi_transports_mobilitat_ca.geojson")

## 3. Limpieza y visualización de datos
Se hará una visualización de todos los datos y se agruparán por indicador

Todos deberán estar en UTM 31

También se hará una limpieza de los datos

Solo se guardará la latitud y la longitud que se obtendrán del atributo geometry

### 3.0. Accesos

In [3]:
accesos.head()

,id,num_acceso,refcat,geometry
0,1,24,2483048DG2028S,POINT (2.06847 41.62147)
1,2,5-11,3959016DG2035N,POINT (2.08500 41.60027)
2,3,78,4377801DG2047N,POINT (2.08993 41.61652)
3,36,4,4281511DG2048S,POINT (2.08948 41.61882)
4,37,6,4281512DG2048S,POINT (2.08947 41.61891)


In [4]:
accesos["latitud"] = accesos["geometry"].y
accesos["longitud"] = accesos["geometry"].x
accesos = accesos.drop(columns=['id', 'num_acceso', 'refcat', 'geometry'])
accesos.head()

,latitud,longitud
0,41.621468,2.068474
1,41.600270,2.085002
2,41.616524,2.089927
3,41.618820,2.089480
4,41.618908,2.089475


### 3.1. Actividades económicas

In [5]:
actividades_economicas.head()

,id,categoria,aa_ida,geometry
0,904,106###PEIXETERIA,ACTIVIDAD_012253,POINT (2.08810 41.61469)
1,1198,106###PEIXETERIA,ACTIVIDAD_012255,POINT (2.08754 41.61611)
2,914,106###PEIXETERIA,ACTIVIDAD_012256,POINT (2.08729 41.61523)
3,1223,110###PERRUQUERIES,ACTIVIDAD_013554,POINT (2.08657 41.61550)
4,1224,110###PERRUQUERIES,ACTIVIDAD_013556,POINT (2.08618 41.61327)


In [6]:
# se eliminan los atributos que innecesarios
actividades_economicas["latitud"] = actividades_economicas["geometry"].y
actividades_economicas["longitud"] = actividades_economicas["geometry"].x
actividades_economicas = actividades_economicas.drop(columns=['id', 'aa_ida', 'geometry'])
actividades_economicas.head()

,categoria,latitud,longitud
0,106###PEIXETERIA,41.614688,2.088102
1,106###PEIXETERIA,41.616111,2.087542
2,106###PEIXETERIA,41.615232,2.087289
3,110###PERRUQUERIES,41.615496,2.086570
4,110###PERRUQUERIES,41.613270,2.086178


#### 3.1.1. Comercios básicos

In [7]:
pescaderias = actividades_economicas[actividades_economicas["categoria"] == "106###PEIXETERIA"]
pescaderias = pescaderias.drop(columns=["categoria"])
pescaderias.reset_index(drop=True, inplace=True)

supermercados = actividades_economicas[actividades_economicas["categoria"] == "129###SUPERMERCATS/HIPERMERCATS"]
supermercados = supermercados.drop(columns=["categoria"])
supermercados.reset_index(drop=True, inplace=True)

fruterias = actividades_economicas[actividades_economicas["categoria"] == "73###FRUITA I VERDURA"]
fruterias = fruterias.drop(columns=["categoria"])
fruterias.reset_index(drop=True, inplace=True)

alimentacion = actividades_economicas[actividades_economicas["categoria"] == "12###ALIMENTACIÓ, BEGUDES I TABAC"]
alimentacion = alimentacion.drop(columns=["categoria"])
alimentacion.reset_index(drop=True, inplace=True)

panaderias = actividades_economicas[actividades_economicas["categoria"] == "71###FORNS I PASTISSERIES"]
panaderias = panaderias.drop(columns=["categoria"])
panaderias.reset_index(drop=True, inplace=True)

carnicerias = actividades_economicas[actividades_economicas["categoria"] == "37###CARNISSERIA"]
carnicerias = carnicerias.drop(columns=["categoria"])
carnicerias.reset_index(drop=True, inplace=True)

alimentacion_no_especializada = actividades_economicas[
    actividades_economicas["categoria"] == "34###BOTIGA D'ALIMENTACIÓ NO ESPECIALITZADA"]
alimentacion_no_especializada = alimentacion_no_especializada.drop(columns=["categoria"])
alimentacion_no_especializada.reset_index(drop=True, inplace=True)

peluquerias = actividades_economicas[actividades_economicas["categoria"] == "110###PERRUQUERIES"]
peluquerias = peluquerias.drop(columns=["categoria"])
peluquerias.reset_index(drop=True, inplace=True)

In [8]:
servicios["latitud"] = servicios["geometry"].y
servicios["longitud"] = servicios["geometry"].x
servicios = servicios.drop(columns=["f_id", "poi_category", "poi_type", "poi_adreca", "poi_telefon", "poi_web", "poi_horari", 
                                "poi_observacions", "tipo_poi", "categoria", "idioma", "atributo", "geometry"])

mercado_municipal = servicios[servicios["poi_nom"] == "Mercat municipal"]
mercado_municipal = mercado_municipal.drop(columns=["poi_nom"])
mercado_municipal.reset_index(drop=True, inplace=True)

mercado_ambulante = servicios[servicios["poi_nom"] == "Mercat ambulant"]
mercado_ambulante = mercado_ambulante.drop(columns=["poi_nom"])
mercado_ambulante.reset_index(drop=True, inplace=True)

#### 3.1.2 Comercios secundarios

In [9]:
electricistas_fontaneros = actividades_economicas[actividades_economicas["categoria"] == "58###ELECTRICISTA I LAMPISTA"]
electricistas_fontaneros = electricistas_fontaneros.drop(columns=["categoria"])
electricistas_fontaneros.reset_index(drop=True, inplace=True)

ferreterias = actividades_economicas[actividades_economicas["categoria"] == "68###FERRETERIA/COPIA CLAUS/CERRAJERIA"]
ferreterias = ferreterias.drop(columns=["categoria"])
ferreterias.reset_index(drop=True, inplace=True)

electrodomesticos = actividades_economicas[actividades_economicas["categoria"] == "59###ELECTRODOMÈSTICS"]
electrodomesticos = electrodomesticos.drop(columns=["categoria"])
electrodomesticos.reset_index(drop=True, inplace=True)

muebles = actividades_economicas[actividades_economicas["categoria"] == "99###MOBLES I DECORACIÓ"]
muebles = muebles.drop(columns=["categoria"])
muebles.reset_index(drop=True, inplace=True)

tiendas_ropa = actividades_economicas[actividades_economicas["categoria"] == "121###ROBA"]
tiendas_ropa = tiendas_ropa.drop(columns=["categoria"])
tiendas_ropa.reset_index(drop=True, inplace=True)

floristerias = actividades_economicas[actividades_economicas["categoria"] == "99###MOBLES I DECORACIÓ"]
floristerias = floristerias.drop(columns=["categoria"])
floristerias.reset_index(drop=True, inplace=True)

zapaterias = actividades_economicas[actividades_economicas["categoria"] == "122###SABATERIA"]
zapaterias = zapaterias.drop(columns=["categoria"])
zapaterias.reset_index(drop=True, inplace=True)

lavanderias = actividades_economicas[actividades_economicas["categoria"] == "36###BUGADERIA"]
lavanderias = lavanderias.drop(columns=["categoria"])
lavanderias.reset_index(drop=True, inplace=True)

mecanicos = actividades_economicas[actividades_economicas["categoria"] == "133###VENDA I REPARACIÓ DE VEHÍCLES DE MOTOR"]
mecanicos = mecanicos.drop(columns=["categoria"])
mecanicos.reset_index(drop=True, inplace=True)

herboristerias = actividades_economicas[actividades_economicas["categoria"] == "51###DIETÈTICA I HERBORISTERIA"]
herboristerias = herboristerias.drop(columns=["categoria"])
herboristerias.reset_index(drop=True, inplace=True)

droguerias = actividades_economicas[actividades_economicas["categoria"] == "54###DROGUERIA - PERFUMERIA"]
droguerias = droguerias.drop(columns=["categoria"])
droguerias.reset_index(drop=True, inplace=True)

tiendas_deportivas = actividades_economicas[actividades_economicas["categoria"] == "87###JOQUINES I ARTICLES D'ESPORT"]
tiendas_deportivas = tiendas_deportivas.drop(columns=["categoria"])
tiendas_deportivas.reset_index(drop=True, inplace=True)

basureros = actividades_economicas[actividades_economicas["categoria"] == "31###BASARS"]
basureros = basureros.drop(columns=["categoria"])
basureros.reset_index(drop=True, inplace=True)

### 3.2. Ocio

#### 3.2.1 Lugares de ocio

In [10]:
ocio.head()

,f_id,poi_category,poi_type,poi_nom,poi_adreca,poi_telefon,poi_horari,poi_observacions,tipo_poi,categoria,idioma,atributo,geometry
0,152,Llocs d'interès,Llocs d'interès,Castell de Castellar,NaN,NaN,NaN,NaN,AAAPAAAG0906,AAAPAAAG0594,ca-ES,NaN,POINT (2.07466 41.60574)
1,168,Llocs d'interès,Llocs d'interès,Camí Fonts Calents,,,,,AAAPAAAG0906,AAAPAAAG0594,ca-ES,,POINT (2.07682 41.62013)
2,200,Llocs d'interès,Àrees d'esbarjo,Àrea d'esbarjo Castellar Vell,,http://www.castellarvalles.cat/14374/equipamen...,,,AAAPAABX4467,AAAPAAAG0594,ca-ES,,POINT (2.07300 41.60193)
3,203,Llocs d'interès,Àrees d'esbarjo,Àrea d'esbarjo de les Arenes,"Ctra. de Sant Llorenç, B124 km. 12,3",619 330 827 / 93 714 84 06,,http://www.castellarvalles.cat/31891/descripti...,AAAPAABX4467,AAAPAAAG0594,ca-ES,AAAPAABG8559,POINT (2.05832 41.63930)
4,204,Llocs d'interès,Llocs d'interès,Pont Vell,"Carretera de Terrassa, km 26,7",,,,AAAPAAAG0906,AAAPAAAG0594,ca-ES,,POINT (2.07832 41.61283)


In [11]:
ocio["latitud"] = ocio["geometry"].y
ocio["longitud"] = ocio["geometry"].x
ocio = ocio.drop(columns=["f_id", "poi_nom", "poi_category", "poi_adreca", "poi_telefon", "poi_horari", "poi_observacions", "tipo_poi",
                          "categoria", "idioma", "atributo", "geometry"])
ocio.poi_type.value_counts()

Llocs d'interès    13
Àrees d'esbarjo     2
Name: poi_type, dtype: int64

In [12]:
lugares_interes = ocio[ocio["poi_type"] == "Llocs d'interès"]
lugares_interes = lugares_interes.drop(columns=["poi_type"])
lugares_interes.reset_index(drop=True, inplace=True)
lugares_interes.head()

,latitud,longitud
0,41.605744,2.074657
1,41.620130,2.076824
2,41.612832,2.078316
3,41.595795,2.092836
4,41.613253,2.084540


In [13]:
bares = actividades_economicas[actividades_economicas["categoria"] == "30###BARS I CAFETERIES"]
bares = bares.drop(columns=["categoria"])
bares.reset_index(drop=True, inplace=True)

restaurantes = actividades_economicas[actividades_economicas["categoria"] == "120###RESTAURANTS"]
restaurantes = restaurantes.drop(columns=["categoria"])
restaurantes.reset_index(drop=True, inplace=True)

#### 3.2.2 Áreas de descanso

In [14]:
areas_descanso = ocio[ocio["poi_type"] == "Àrees d'esbarjo"]
areas_descanso = areas_descanso.drop(columns=["poi_type"])
areas_descanso.reset_index(drop=True, inplace=True)
areas_descanso.head()

,latitud,longitud
0,41.601930,2.072996
1,41.639303,2.058323


### 3.3. Cultura

In [15]:
cultura.head(2)

,f_id,poi_category,poi_type,poi_nom,poi_adreca,poi_telefon,poi_web,poi_horari,poi_observacions,tipo_poi,categoria,idioma,atributo,geometry
0,37,Cultura i oci,Biblioteques i arxius,Arxiu d'Història de Castellar,"C. de la Mina, 9-11",93 714 40 88,http://www.castellarvalles.cat/14374/equipamen...,,arxiuhistoriacastellar@gmail.com,AAAPAAAG0610,AAAPAAAG0590,ca-ES,AAAPAABY3109,POINT (2.08938 41.61810)
1,38,Cultura i oci,Biblioteques i arxius,Biblioteca Antoni Tort,"C. de Sala Boadella, 6",93 714 47 35,http://www.castellarvalles.cat/14374/equipamen...,,http://bibliotecaantonitort.wordpress.com/,AAAPAAAG0610,AAAPAAAG0590,ca-ES,AAAPAABY2763,POINT (2.08784 41.61535)


In [16]:
cultura["latitud"] = cultura["geometry"].y
cultura["longitud"] = cultura["geometry"].x
cultura = cultura.drop(columns=["f_id", "poi_nom", "poi_category", "poi_adreca", "poi_telefon", "poi_web", "poi_horari", 
                                "poi_observacions", "tipo_poi", "categoria", "idioma", "atributo", "geometry"])
cultura.poi_type.value_counts()

Parcs, places i jardins         15
Espais socials i culturals       9
Exposicions i galeries d'art     6
Biblioteques i arxius            3
Name: poi_type, dtype: int64

In [17]:
parques = cultura[cultura["poi_type"] == "Parcs, places i jardins"]
parques = parques.drop(columns=["poi_type"])
parques.reset_index(drop=True, inplace=True)

espacios_culturales = cultura[cultura["poi_type"] == "Espais socials i culturals"]
espacios_culturales = espacios_culturales.drop(columns=["poi_type"])
espacios_culturales.reset_index(drop=True, inplace=True)

exposiciones = cultura[cultura["poi_type"] == "Exposicions i galeries d'art"]
exposiciones = exposiciones.drop(columns=["poi_type"])
exposiciones.reset_index(drop=True, inplace=True)

bibliotecas = cultura[cultura["poi_type"] == "Biblioteques i arxius"]
bibliotecas = bibliotecas.drop(columns=["poi_type"])
bibliotecas.reset_index(drop=True, inplace=True)

### 3.4. Educación

In [18]:
educacion.head(2)

,f_id,poi_category,poi_type,poi_nom,poi_adreca,poi_telefon,poi_web,tipo_poi,categoria,idioma,atributo,geometry
0,46,Educació,Altres,Escola municipal d'Adults,"C. dels Pedrissos, 9, Local",93 714 83 55,http://www.castellarvalles.cat/14374/equipamen...,AAAPAAAG0617,AAAPAAAG0591,ca-ES,AAAPAAAG1203,POINT (2.08532 41.61079)
1,47,Educació,Altres,Escola de Música Torre Balada,"C. de Caldes, 56",93 714 43 22,http://www.castellarvalles.cat/14374/equipamen...,AAAPAAAG0617,AAAPAAAG0591,ca-ES,AAAPAAAG1204,POINT (2.09104 41.61752)


In [19]:
educacion["latitud"] = educacion["geometry"].y
educacion["longitud"] = educacion["geometry"].x
educacion = educacion.drop(columns=["f_id", "poi_nom", "poi_category", "poi_adreca", "poi_telefon", "poi_web", "tipo_poi", "categoria", 
                                    "idioma", "atributo", "geometry"])
educacion.poi_type.value_counts()

Educació infantil i primària    9
Escoles bressol                 8
Educació secundària             4
Altres                          3
Name: poi_type, dtype: int64

In [20]:
colegios = educacion[educacion["poi_type"] == "Educació infantil i primària"]
colegios = colegios.drop(columns=["poi_type"])
colegios.reset_index(drop=True, inplace=True)

guarderias = educacion[educacion["poi_type"] == "Escoles bressol"]
guarderias = guarderias.drop(columns=["poi_type"])
guarderias.reset_index(drop=True, inplace=True)

institutos = educacion[educacion["poi_type"] == "Educació secundària"]
institutos = institutos.drop(columns=["poi_type"])
institutos.reset_index(drop=True, inplace=True)

otros_centros = educacion[educacion["poi_type"] == "Altres"]
otros_centros = otros_centros.drop(columns=["poi_type"])
otros_centros.reset_index(drop=True, inplace=True)

### 3.5. Sanidad

In [21]:
sanidad.head(2)

,f_id,poi_category,poi_type,poi_nom,poi_adreca,poi_telefon,poi_web,poi_horari,poi_observacions,tipo_poi,categoria,idioma,atributo,geometry
0,18,Sanitat i salut,CAP,Centre d'Atenció Primària,"C. de Ripollet, 30",93 728 44 44,http://www.castellarvalles.cat/directori/?dire...,,http://www.castellarvalles.cat/14374/equipamen...,AAAPAAAA9452,AAAPAAAA9435,ca-ES,AAAPAABY5795,POINT (2.09073 41.61080)
1,22,Sanitat i salut,Farmàcies,Farmàcia Casanovas,"Av. de Sant Esteve, 3",93 747 49 56,http://www.castellarvalles.cat/14374/equipamen...,,http://www.farmaciacasanovas.cat,AAAPAAAG0606,AAAPAAAA9435,ca-ES,AAAPAABY6673,POINT (2.08664 41.61312)


In [22]:
sanidad["latitud"] = sanidad["geometry"].y
sanidad["longitud"] = sanidad["geometry"].x
sanidad = sanidad.drop(columns=["f_id", "poi_nom", "poi_category", "poi_adreca", "poi_telefon", "poi_web", "poi_horari", 
                                "poi_observacions", "tipo_poi", "categoria", "idioma", "atributo", "geometry"])
sanidad.poi_type.value_counts()

desfibril·ladors    17
Farmàcies            9
Residències          5
CAP                  1
Name: poi_type, dtype: int64

In [23]:
farmacias = sanidad[sanidad["poi_type"] == "Farmàcies"]
farmacias = farmacias.drop(columns=["poi_type"])
farmacias.reset_index(drop=True, inplace=True)

residencias = sanidad[sanidad["poi_type"] == "Residències"]
residencias = residencias.drop(columns=["poi_type"])
residencias.reset_index(drop=True, inplace=True)

cap = sanidad[sanidad["poi_type"] == "CAP"]
cap = cap.drop(columns=["poi_type"])
cap.reset_index(drop=True, inplace=True)

cruz_roja = servicios[servicios["poi_nom"] == "Creu Roja"]
cruz_roja = cruz_roja.drop(columns=["poi_nom"])
cruz_roja.reset_index(drop=True, inplace=True)

### 3.6. Deporte

In [24]:
deporte.head(2)

,f_id,poi_category,poi_type,poi_nom,poi_adreca,poi_telefon,poi_web,poi_observacions,tipo_poi,categoria,idioma,atributo,geometry
0,60,Instal·lacions esportives,Esports,Pistes d'Atletisme,"C. de la Garrotxa, 1",659 59 17 38 - 93 714 73 54 (Club Atletic Cast...,http://www.castellarvalles.cat/14374/equipamen...,NaN,AAAPAAAG0621,AAAPAAAG0592,ca-ES,NaN,POINT (2.09294 41.60505)
1,61,Instal·lacions esportives,Esports,Casino del Racó,"C. de Brutau, 19. Sant Feliu del Racó",699 03 92 67,http://www.castellarvalles.cat/14374/equipamen...,http://www.casinodelraco.cat,AAAPAAAG0621,AAAPAAAG0592,ca-ES,AAAPAABY4987,POINT (2.06871 41.62155)


In [25]:
deporte["latitud"] = deporte["geometry"].y
deporte["longitud"] = deporte["geometry"].x
deporte = deporte.drop(columns=["f_id", "poi_category", "poi_type", "poi_nom", "poi_adreca", "poi_telefon", "poi_web", 
                                "poi_observacions", "tipo_poi", "categoria", "idioma", "atributo", "geometry"])

### 3.7. Movilidad

In [26]:
movilidad.head(2)

,f_id,poi_category,poi_type,poi_nom,poi_adreca,poi_telefon,poi_horari,poi_observacions,tipo_poi,categoria,idioma,atributo,geometry
0,206,Transports,Aparcaments,Aparcament públic plaça major,NaN,NaN,NaN,NaN,AAAPAABX4462,AAAPAABX4445,ca-ES,NaN,POINT (2.08798 41.61602)
1,207,Transports,Taxi,Parada de taxis,,,,,AAAPAABX4457,AAAPAABX4445,ca-ES,,POINT (2.08609 41.61557)


In [27]:
movilidad["latitud"] = movilidad["geometry"].y
movilidad["longitud"] = movilidad["geometry"].x
movilidad = movilidad.drop(columns=["f_id", "poi_category", "poi_adreca", "poi_telefon", "poi_horari", "poi_observacions", 
                                  "tipo_poi", "categoria", "idioma", "atributo"])
movilidad.poi_type.value_counts()

Places per a persones amb mobilitat reduïda    109
Autobús                                         56
Càrrega i descàrrega                            35
Zones blaves                                    18
Aparcaments                                     12
Transport a la demanda                          10
Aparcament Farmàcia                              8
Benzineres                                       6
Reserva Assistencial                             5
Zones Vermelles                                  2
Taxi                                             1
Punt Elèctric                                    1
Name: poi_type, dtype: int64

In [28]:
autobus = movilidad[movilidad["poi_type"] == "Autobús"]
autobus = autobus.drop(columns=["poi_type"])
autobus.reset_index(drop=True, inplace=True)

taxi = movilidad[movilidad["poi_type"] == "Taxi"]
taxi = taxi.drop(columns=["poi_type"])
taxi.reset_index(drop=True, inplace=True)

transporte_demanda = movilidad[movilidad["poi_type"] == "Transport a la demanda"]
transporte_demanda = transporte_demanda.drop(columns=["poi_type"])
transporte_demanda.reset_index(drop=True, inplace=True)

### 3.8. Servicios públicos

In [29]:
servicios.poi_nom.value_counts()

Correus                  1
Cementiri municipal      1
Tanatori municipal       1
Notaria                  1
Deixalleria municipal    1
Depuradora municipal     1
Mercat municipal         1
Mercat ambulant          1
Creu Roja                1
Name: poi_nom, dtype: int64

In [30]:
correos = servicios[servicios["poi_nom"] == "Correus"]
correos = correos.drop(columns=["poi_nom"])
correos.reset_index(drop=True, inplace=True)

notaria = servicios[servicios["poi_nom"] == "Notaria"]
notaria = notaria.drop(columns=["poi_nom"])
notaria.reset_index(drop=True, inplace=True)

In [31]:
administracion.head(2)

,f_id,poi_category,poi_type,poi_nom,poi_adreca,poi_telefon,poi_web,poi_horari,poi_observacions,tipo_poi,categoria,idioma,atributo,geometry
0,1,Administració,Dependències administratives,Ajuntament de Castellar - Palau Tolrà,"Pg. Tolrà, 1",937144040,http://www.castellarvalles.cat/14374/equipamen...,"de dilluns a divendres, de 8 a 15 h",ajuntament@castellarvalles.cat,AAAPAAAG0558,AAAPAAAG0556,ca-ES,AAAPAABX8574,POINT (2.08781 41.61847)
1,3,Administració,Dependències administratives,Safareigs de la Baixada del Palau,"C. de la Baixada del Palau, 1",93 714 42 06,http://www.castellarvalles.cat/14374/equipamen...,,dpeio@castellarvalles.cat,AAAPAAAG0558,AAAPAAAG0556,ca-ES,AAAPAABY2706,POINT (2.08674 41.61908)


In [32]:
administracion["latitud"] = administracion["geometry"].y
administracion["longitud"] = administracion["geometry"].x

administracion = administracion.drop(columns=["f_id", "poi_category", "poi_type", "poi_adreca", "poi_telefon", "poi_web", "poi_horari", 
                                              "poi_observacions", "tipo_poi", "categoria", "idioma", "atributo"])

ayuntamiento = administracion[administracion["poi_nom"] == "Ajuntament de Castellar - Palau Tolrà"]
ayuntamiento = ayuntamiento.drop(columns=["poi_nom"])
ayuntamiento.reset_index(drop=True, inplace=True)

## 4. Cálculo de distancias
En primer lugar se guardan todas las variables en un array para poder acceder a ellas más fácilmente

In [33]:
localizaciones = [supermercados, pescaderias, alimentacion, panaderias, carnicerias, alimentacion_no_especializada, 
                  peluquerias, mercado_municipal, mercado_ambulante, electricistas_fontaneros, ferreterias, 
                  electrodomesticos, muebles, tiendas_ropa, floristerias, lugares_interes, bares, restaurantes, 
                  areas_descanso, parques, espacios_culturales, exposiciones, bibliotecas, colegios, guarderias, institutos,
                  otros_centros, farmacias, residencias, cap, cruz_roja, deporte, autobus, taxi, transporte_demanda, 
                  correos, notaria, ayuntamiento]

a continuación se define la función que calcula la distancia mínima

In [34]:
# función que calcula la distancia mínima
def distancia_minima(distancia1, distancia2):
    if distancia1 < distancia2:
        distancia_min = distancia1
    else:
        distancia_min = distancia2
    return distancia_min

Se calcula la distancia euclídea utilizando la función haversine

In [35]:
# añadir las localizaciones al dataset de accesos, dándoles un valor inicial de 0
for i in range(0, len(localizaciones)):
    accesos[i] = 0

# cálculo de distancia euclídea
for acceso in accesos.index:
    for i in range(0, len(localizaciones)):
        # se da una distancia inicial muy alta para que esta sea superada con el primer punto
        dist_minima = 10000.0
        for localizacion in localizaciones[i].index:
            # calcula la distancia
            dist = hs.haversine((accesos.latitud[acceso], accesos.longitud[acceso]), 
                                (localizaciones[i].latitud[localizacion], localizaciones[i].longitud[localizacion])) 
            # compara la distancia y guarda la menor
            dist_minima = distancia_minima(dist_minima, dist)
            accesos[i][acceso] = dist_minima
            
accesos.head()

,latitud,longitud,0,1,2,3,4,5,6,7,...,28,29,30,31,32,33,34,35,36,37
0,41.621468,2.068474,1.679175,1.693280,1.611722,1.448692,1.488891,1.421424,1.489582,1.702200,...,1.522930,2.197693,1.881989,0.021322,0.169901,1.604619,1.580469,1.815886,1.748584,1.641247
1,41.600270,2.085002,0.252006,1.623818,0.042027,0.337488,1.393839,0.247212,0.841635,1.784451,...,1.495713,1.263733,1.654647,0.582840,0.090801,1.704206,0.090903,1.635855,1.718108,2.036893
2,41.616524,2.089927,0.210746,0.203514,0.145543,0.024980,0.119155,0.122335,0.054018,0.189121,...,0.312552,0.640230,0.198843,0.283565,0.114560,0.335685,0.346609,0.233913,0.194862,0.278881
3,41.618820,2.089480,0.334710,0.341637,0.125297,0.085279,0.331973,0.292724,0.111902,0.327856,...,0.442415,0.898065,0.447123,0.530608,0.241876,0.457735,0.453036,0.460310,0.383579,0.144598
4,41.618908,2.089475,0.342875,0.350030,0.134769,0.092068,0.341433,0.299549,0.105774,0.336353,...,0.448416,0.907758,0.456810,0.536240,0.250028,0.465168,0.460097,0.469750,0.392627,0.147120


La distancia está en kilómetros, por lo que hay que pasarla a metros

In [37]:
accesos_euc = accesos.copy()

for columna in accesos.iloc[:, 2:].columns:
    accesos_euc[columna] = accesos[columna] * 1000
    
accesos_euc.to_csv('Resultados/accesos_distancia_euc.csv')

accesos_euc.head()